In [2]:
# Install Kubeflow Pipelines SDK
!PIP_DISABLE_PIP_VERSION_CHECK=1 pip3 install 'kfp>=0.1.32.2' --quiet

In [ ]:
import kfp
# Initializing the client
client = kfp.Client()

# ! Use kfp.Client(host='https://xxxxx.notebooks.googleusercontent.com/') if working from GCP notebooks (or local notebooks)

In [3]:
build_image_component_text = '''
name: Build container image
inputs:
- {name: Context archive URI, type: URI}
- {name: Target image name, type: String}
outputs:
- {name: Image digest, type: String}
implementation:
  container:
    image: gcr.io/kaniko-project/executor@sha256:78d44ec4e9cb5545d7f85c1924695c89503ded86a59f92c7ae658afa3cff5400
    command:
    - /kaniko/executor
    - --cache=true
    - --dockerfile
    - Dockerfile
    - --context
    - inputValue: Context archive URI
    - --destination
    - inputValue: Target image name
    - --digest-file
    - /tmp/digest.txt
    fileOutputs:
      Image digest: /tmp/digest.txt
'''

build_image_op = kfp.components.load_component_from_text(build_image_component_text)

In [4]:
git_clone_component_text = '''
name: Git clone
inputs:
- {name: Repo URI, type: URI}
- {name: Branch, type: String, default: 'master'}
outputs:
- {name: Repo dir, type: Directory}
implementation:
  container:
    image: alpine/git
    command:
    - git
    - clone
    - --depth=1
    - --branch
    - inputValue: Branch
    - inputValue: Repo URI
    - outputPath: Repo dir
'''

import kfp
git_clone_op = kfp.components.load_component_from_text(git_clone_component_text)

In [5]:
get_subdir_component_text = '''
name: Get subdirectory
description: Get subdirectory items (files or directories).
inputs:
- {name: Directory, type: Directory}
- {name: Subpath, type: String}
outputs:
- {name: Subdir, type: Directory}
implementation:
  container:
    image: alpine
    command:
    - sh
    - -ex
    - -c
    - |
      mkdir -p "$(dirname "$2")"
      cp -r "$0/$1" "$2"
    - inputPath: Directory
    - inputValue: Subpath
    - outputPath: Subdir
'''
get_subdir_op = kfp.components.load_component_from_text(get_subdir_component_text)

In [6]:
tar_gzip_component_text = '''
name: Compress directory using TAR GZIP
inputs:
- {name: Data, type: Directory}
outputs:
- {name: Gzipped data, type: GzippedTar}
implementation:
  container:
    image: alpine
    command:
    - sh
    - -ex
    - -c
    - |
      mkdir -p "$(dirname "$1")"
      cd "$0"
      tar c -z -f "$1" .
    - inputPath: Data
    - outputPath: Gzipped data
'''
tar_gzip_op = kfp.components.load_component_from_text(tar_gzip_component_text)

In [7]:
upload_to_gcs_component_text = '''
name: Upload to GCS
inputs:
- {name: Data}
- {name: GCS path, type: URI}
outputs:
- {name: GCS path, type: URI}
implementation:
  container:
    image: google/cloud-sdk
    command:
    - sh
    - -ex
    - -c
    - |
      gcloud auth activate-service-account --key-file="${GOOGLE_APPLICATION_CREDENTIALS}"
      gsutil cp "$0" "$1"
      mkdir -p "$(dirname "$2")"
      echo "$1" > "$2"
    - inputPath: Data
    - inputValue: GCS path
    - outputPath: GCS path
'''
upload_to_gcs_op = kfp.components.load_component_from_text(upload_to_gcs_component_text)

In [8]:
print_component_text = '''
name: Print text
inputs:
- {name: Text}
implementation:
  container:
    image: alpine
    command:
    - cat
    - inputPath: Text
'''
print_op = kfp.components.load_component_from_text(print_component_text)

In [56]:
def my_pipeline2():
    git_clone_task = git_clone_op(
        repo_uri='https://github.com/kubeflow/pipelines.git',
    )
    get_subdir_task = get_subdir_op(
        directory=git_clone_task.output,
        subpath='components/sample/keras/train_classifier',
    )
    tar_gzip_task = tar_gzip_op(
        data=get_subdir_task.output,
    )
    upload_to_gcs_task = upload_to_gcs_op(
        data=tar_gzip_task.output,
        gcs_path='gs://avolkov/tmp_git_pipeline/tmp123.tgz',
    )
    build_image_task = build_image_op(
        context_archive_uri=upload_to_gcs_task.output,
        target_image_name='gcr.io/avolkov-31337/tmp1:tag1',
    )
    print_op(
        build_image_task.outputs['image_digest']
    )
    

from kfp import gcp
pipeline_conf=kfp.dsl.PipelineConf()
pipeline_conf.add_op_transformer(gcp.use_gcp_secret('user-gcp-sa'))

client.create_run_from_pipeline_func(my_pipeline2, arguments={}, pipeline_conf=pipeline_conf)

<kfp._client.Client.create_run_from_pipeline_package.<locals>.RunPipelineResult at 0x7f8cb409fa58>

In [9]:
build_image_component_text = '''
name: Build container image
inputs:
- {name: Context}
- {name: Target image name, type: String}
outputs:
- {name: Image digest, type: String}
implementation:
  container:
    image: gcr.io/kaniko-project/executor@sha256:78d44ec4e9cb5545d7f85c1924695c89503ded86a59f92c7ae658afa3cff5400
    command:
    - /kaniko/executor
    - --cache=true
    - --dockerfile
    - Dockerfile
    - --context
    - inputPath: Context
    - --destination
    - inputValue: Target image name
    - --digest-file
    - /tmp/digest.txt
    fileOutputs:
      Image digest: /tmp/digest.txt
'''

build_image_op = kfp.components.load_component_from_text(build_image_component_text)

In [12]:
def my_pipeline2():
    git_clone_task = git_clone_op(
        repo_uri='https://github.com/kubeflow/pipelines.git',
    )
    get_subdir_task = get_subdir_op(
        directory=git_clone_task.output,
        subpath='components/sample/keras/train_classifier',
    )
    build_image_task = build_image_op(
        context=get_subdir_task.output,
        target_image_name='gcr.io/avolkov-31337/tmp1:tag1',
    )
    print_op(
        build_image_task.outputs['image_digest']
    )
    

from kfp import gcp
pipeline_conf=kfp.dsl.PipelineConf()
pipeline_conf.add_op_transformer(gcp.use_gcp_secret('user-gcp-sa'))

client.create_run_from_pipeline_func(my_pipeline2, arguments={}, pipeline_conf=pipeline_conf)

RunPipelineResult(run_id=b0faff0d-f778-11e9-93ae-42010a800216)